In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir
from datetime import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
root_folder = 'drive/My Drive/LaTrobe/Projects/Accelerometer/OA activity data_La Trobe'
overlap_data_file = join(root_folder, 'Analytics_Scripts/metadata/record_avaiability.xlsx')
hkoa_data_file = join(root_folder, 'anthro.xlsx')
oa_data_folder = join(root_folder, 'sleep_oa')

In [ ]:
main_df = pd.read_excel(overlap_data_file)

In [ ]:
oa_data_files = { "OA{}".format(int(f.split('_')[-1].replace('.csv', ''))): f for f in listdir(oa_data_folder) if isfile(join(oa_data_folder, f))}

## Combine HOA, KOA and metadata

In [ ]:
main_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d
1,1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d,2019-06-24 14:00:00,2019-07-01 07:59:59,161h 59m 59d,161h 59m 59d
2,2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d,2019-07-01 14:00:00,2019-07-08 07:59:59,161h 59m 59d,161h 59m 59d
3,3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d,2019-07-01 16:00:00,2019-07-08 07:59:59,159h 59m 59d,159h 59m 59d
4,4,HOA13,2019-07-08 16:59:47,2019-07-15 10:12:35,161h 12m 48d,NaT,NaT,NaN,NaN


In [ ]:
hkoa_data_df.head()

,pid,age_yr,sex,BMI,bodymass_kg,av_inbedtime,av_outofbed_time,AG_data,AP_data,"Side_TKA (1=L,2=R)"
0,HOA1,52.0,female,76.86261,88.0,22:00:00,06:00:00,NaN,NaN,NaN
1,HOA2,69.0,female,39.65711,99.0,20:00:00,04:30:00,NaN,NaN,NaN
2,HOA3,36.0,female,41.15226,75.0,01:00:00,09:00:00,NaN,NaN,NaN
3,HOA4,42.0,female,46.43327,113.0,22:30:00,09:00:00,NaN,NaN,NaN
4,HOA5,68.0,female,34.76563,89.0,20:00:00,05:30:00,NaN,NaN,NaN


In [ ]:
main_df_merged = pd.merge(main_df, hkoa_data_df, left_on='Participant', right_on='pid', how='left')

In [ ]:
main_df_merged.tail()

## Duplicate OA data with daily sleep times

In [ ]:
oa_data_df = None

for key, fname in tqdm(oa_data_files.items()):

  temp_df = pd.read_csv(join(oa_data_folder, fname))
  temp_df['pid'] = key

  if oa_data_df is not None:
    oa_data_df = oa_data_df.append(temp_df)
  else:
    oa_data_df = temp_df

# Update datetime
oa_data_df['SLEEP_start'] = pd.to_datetime(oa_data_df['SLEEP_start'], format='%d%b%y:%H:%M:%S')
oa_data_df['SLEEP_finish'] = pd.to_datetime(oa_data_df['SLEEP_finish'], format='%d%b%y:%H:%M:%S')

oa_data_df = oa_data_df.rename(columns={'SLEEP_start': 'av_inbedtime', 'SLEEP_finish': 'av_outofbed_time'})

100%|██████████| 88/88 [00:00<00:00, 167.61it/s]


In [ ]:
merged_df = pd.merge(main_df_merged, oa_data_df, left_on='pid', right_on='pid', how='left')

In [ ]:
merged_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,pid,age_yr,sex,BMI,bodymass_kg,av_inbedtime_x,av_outofbed_time_x,AG_data,AP_data,"Side_TKA (1=L,2=R)",av_inbedtime_y,av_outofbed_time_y,slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,HOA1,52.0,female,76.86261,88.0,22:00:00,06:00:00,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d,2019-06-24 14:00:00,2019-07-01 07:59:59,161h 59m 59d,161h 59m 59d,HOA10,51.0,female,42.41690,98.0,21:00:00,09:00:00,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d,2019-07-01 14:00:00,2019-07-08 07:59:59,161h 59m 59d,161h 59m 59d,HOA11,56.0,male,30.09143,78.0,20:00:00,06:00:00,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d,2019-07-01 16:00:00,2019-07-08 07:59:59,159h 59m 59d,159h 59m 59d,HOA12,61.0,female,27.43484,72.0,20:00:00,05:00:00,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,4,HOA13,2019-07-08 16:59:47,2019-07-15 10:12:35,161h 12m 48d,NaT,NaT,NaN,NaN,HOA13,56.0,female,36.73095,100.0,20:00:00,05:00:00,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df['av_inbedtime'] = merged_df['av_inbedtime_x'].astype(str) + merged_df['av_inbedtime_y'].astype(str)
merged_df['av_inbedtime'] = merged_df['av_inbedtime'].str.replace('nan', '')
merged_df['av_inbedtime'] = merged_df['av_inbedtime'].str.replace('NaT', '')

merged_df['av_outofbed_time'] = merged_df['av_outofbed_time_x'].astype(str) + merged_df['av_outofbed_time_y'].astype(str)
merged_df['av_outofbed_time'] = merged_df['av_outofbed_time'].str.replace('nan', '')
merged_df['av_outofbed_time'] = merged_df['av_outofbed_time'].str.replace('NaT', '')

del merged_df['av_inbedtime_x']
del merged_df['av_inbedtime_y']
del merged_df['av_outofbed_time_x']
del merged_df['av_outofbed_time_y']
del merged_df['pid']

In [ ]:
merged_df.tail(40)

In [ ]:
merged_df.to_excel('sleep_time.xlsx', index=None)